In [1]:
import pandas as pd
import orjson as json
import tqdm
from features import general_features

In [2]:
def general_features_df(df):
    df = df[['game_time', 'game_mode', 'lobby_type']]
    df['game_phase'] = pd.cut(df.game_time / 60, bins=[-3, 13, 26, 100],
                                right=False, include_lowest=True,
                                labels=['early_game', 'mid_game', 'late_game'])
    df.game_mode = df.game_mode.map(game_mode_map)
    df.lobby_type = df.lobby_type.map(lobby_type_map)
    df.game_mode[df.game_mode == "Captains Draft"] = "Captains Mode"
    df.game_mode[df.game_mode == "Least Played"] = "Random Draft"
    df.game_mode[df.game_mode == "All Random"] = "Random Draft"
    return df

In [3]:
train_features = []
for line in tqdm.tqdm(open('../data/train_matches.jsonl')):
    game = json.loads(line)
    train_features.append(general_features(game))

39675it [01:06, 600.47it/s]


In [4]:
test_features = []
for line in tqdm.tqdm(open('../data/test_matches.jsonl')):
    game = json.loads(line)
    test_features.append(general_features(game))

10000it [00:13, 743.00it/s]


In [5]:
train_features = pd.DataFrame(train_features)
test_features = pd.DataFrame(test_features)

In [6]:
train_features.to_csv('../features/gen_train.csv', index=False)
test_features.to_csv('../features/gen_test.csv', index=False)